In [1]:
import numpy as np
import pandas as pd
#import statsmodels.api as sm
import databricks.koalas as ks
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import time

In [2]:
raw_data_act = ks.read_parquet('activityObf.parquet')

In [3]:
df_act_sorted=raw_data_act.sort_values(by=['user_id','periodo']).reset_index(drop=True) # Ordeno el dataframe de Activity
df_act_wo_duplicates = df_act_sorted.drop_duplicates().reset_index(drop=True) # Saco las filas repetidas


In [9]:
df_act_wo_duplicates2 = ks.DataFrame({'user_id': [1,2,3], 'periodo': ['1-1-1','1-2-1','1-3-1']})

In [15]:
def hist_actividad(df, feature_name, year, histogram = True):
    '''Si histogram == False, devuelve un dataframe con los períodos de actividad y user_id de determinado año'''
    aux = df[(df[feature_name] >= datetime.date(year,1,1)) & (df[feature_name] < datetime.date(year+1,1,1))]  
    #aux = df[(df[feature_name] >= '{}-1-1'.format(year) ) & (df[feature_name] < '{}-1-1'.format(year+1) )] 
    
    if histogram == True:
        plt.hist(aux[feature_name]);        
        plt.title(year, fontsize = 20)
    
    else:
        return aux

def get_month_diff(a, b):
      
      return 12*(a.year - b.year) + (a.month - b.month)       
      
def se_fuga(df):
    
      if (df['user_id'] == df['next_user_id']) and ( get_month_diff(df['next_periodo'], df['periodo']) >= 4) :
          return 1
      #elif (df['user_id'] != df['next_user_id']) and ( get_month_diff(max_date, df['periodo']) >= 4) :
      #    return 0
      else:
          return 0
    
def define_estados(df_aux):
    
      #df2 = df_aux.copy()
      
      aux_userid = list(df_aux['user_id'].to_numpy())#.pop(0)+[0]
      aux_periodo = list(df_aux['periodo'].to_numpy())#.pop(0)+['0-1-1']  
    
      aux_userid.pop(0); aux_periodo.pop(0);
      
      aux_userid = aux_userid+[0]
      aux_periodo = aux_periodo+[]
    
      ks_next = ks.DataFrame({'next_user_id': np.array(aux_userid), 'next_periodo': np.array(aux_periodo)})
    
      df_aux.assign(next_user_id=ks_next['next_user_id'], next_periodo = ks_next['next_periodo'] )
    
      #df2['next_user_id'] = np.array(aux_userid)#np.array(df2['user_id'].shift(-1))
      #df2['next_periodo'] = np.array(aux_periodo)#np.array(df2['periodo'].shift(-1))
      
      return df_aux.applymap(se_fuga, axis=1)

In [16]:
start_time = time.clock()

states = define_estados(df_act_wo_duplicates2);
print((time.clock() - start_time)/60, "minutos")

/home/dran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


AnalysisException: 'Resolved attribute(s) next_periodo#118,next_user_id#117L missing from __index_level_0__#76L,user_id#77L,periodo#78 in operator !Project [__index_level_0__#76L, user_id#77L, periodo#78, next_user_id#117L AS next_user_id#124L, next_periodo#118 AS next_periodo#125].;;\n!Project [__index_level_0__#76L, user_id#77L, periodo#78, next_user_id#117L AS next_user_id#124L, next_periodo#118 AS next_periodo#125]\n+- LogicalRDD [__index_level_0__#76L, user_id#77L, periodo#78], false\n'

In [ ]:
df_pd = df_act_wo_duplicates.head(10).to_pandas()